# 🏗️ Engenharia de Features: Tabelas Globais (Treino)

Gera a visão histórica completa das variáveis exógenas (2019+) para permitir o treinamento robusto.

---

# **Bibliotecas Importantes:**

In [0]:
# Processamento SQL massivo de histórico
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType, LongType, DecimalType, FloatType, ShortType
from src.ingestion.connectors import connect_jdbc, url_jdbc
from src.ingestion.feature_store import salvar_feature_table

In [0]:
ANO = '2019'
DATA_INICIAL = '2019-01-01'

# **Tabelas Globais:**

## **CAMPANHAS:**

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
SELECT
  data,
  COALESCE(`PAIS`, 0)                                   AS campanhas_pais,
  COALESCE(`FESTIVAL DE TRUFAS`, 0)                     AS campanhas_festival_de_trufas,
  COALESCE(`MÊS DO CHOCOLATE`, 0)                       AS campanhas_mes_do_chocolate,
  COALESCE(`LIFESTYLE HOME`, 0)                         AS campanhas_lifestyle_home,
  COALESCE(`BISCOITO`, 0)                               AS campanhas_biscoito,
  COALESCE(`MULHER, MÃES E NAMORADOS`, 0)               AS campanhas_mulher_maes_e_namorados,
  COALESCE(`VERÃO`, 0)                                  AS campanhas_verao,
  COALESCE(`CRIANÇAS, PROFESSORES E HALLOWEEN`, 0)      AS campanhas_criancas_professores_e_halloween,
  COALESCE(`MIAU`, 0)                                   AS campanhas_miau,
  COALESCE(`FESTIVAL DE TABLETES`, 0)                   AS campanhas_festival_de_tabletes,
  COALESCE(`BLESS`, 0)                                  AS campanhas_bless,
  COALESCE(`FESTIVAL LACREME`, 0)                       AS campanhas_festival_lacreme,
  COALESCE(`BYTES`, 0)                                  AS campanhas_bytes,
  COALESCE(`ARRAIÁ E MÊS DO CHOCOLATE`, 0)              AS campanhas_arraia_e_mes_do_chocolate,
  COALESCE(`MONTEBELLO`, 0)                             AS campanhas_montebello,
  COALESCE(`LANÇAMENTOS`, 0)                            AS campanhas_lancamentos,
  COALESCE(`FESTIVAL 9.90`, 0)                          AS campanhas_festival_9_90,
  COALESCE(`FESTIVAL DE PISTACHE`, 0)                   AS campanhas_festival_de_pistache,
  COALESCE(`NATAL`, 0)                                  AS campanhas_natal,
  COALESCE(`PÁSCOA`, 0)                                 AS campanhas_pascoa,
  COALESCE(`GELATO`, 0)                                 AS campanhas_gelato
FROM (
  SELECT data, descricao_campanha
  FROM ds_dev.cvc_val.campanhas
) PIVOT (
  CASE WHEN COUNT(*) >0 THEN 1 ELSE 0 END FOR descricao_campanha IN (
    'PAIS','FESTIVAL DE TRUFAS','MÊS DO CHOCOLATE',
    'LIFESTYLE HOME','BISCOITO','MULHER, MÃES E NAMORADOS',
    'VERÃO','CRIANÇAS, PROFESSORES E HALLOWEEN','MIAU',
    'FESTIVAL DE TABLETES','BLESS','FESTIVAL LACREME','BYTES',
    'ARRAIÁ E MÊS DO CHOCOLATE','MONTEBELLO','LANÇAMENTOS','FESTIVAL 9.90',
    'FESTIVAL DE PISTACHE','NATAL','PÁSCOA','GELATO'
  )
)
WHERE YEAR(data)>={ANO}
ORDER BY data ''').createOrReplaceTempView("campanhas")

## **PROMOÇÕES:**

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f''' 
SELECT   data
        ,COALESCE(`PASTAS`,0)                        AS  promocoes_pastas
        ,COALESCE(`OVOS`,0)                          AS  promocoes_ovos
        ,COALESCE(`MARSHMALLOWS`,0)                  AS  promocoes_marshmallows
        ,COALESCE(`CHOCOARTES`,0)                    AS  promocoes_chocoartes
        ,COALESCE(`BEBIDAS ALCOOLICAS`,0)            AS  promocoes_bebidas_alcoolicasOLICAS
        ,COALESCE(`FIGURAS`,0)                       AS  promocoes_figuras
        ,COALESCE(`CLUSTERS`,0)                      AS  promocoes_clusters
        ,COALESCE(`LIFESTYLE`,0)                     AS  promocoes_lifestyle
        ,COALESCE(`CARTOES`,0)                       AS  promocoes_cartoes
        ,COALESCE(`OVINHOS`,0)                       AS  promocoes_ovinhos
        ,COALESCE(`PIRULITOS`,0)                     AS  promocoes_pirulitos
        ,COALESCE(`ESFERAS`,0)                       AS  promocoes_esferas
        ,COALESCE(`GRANEIS`,0)                       AS  promocoes_graneis
        ,COALESCE(`FLORES`,0)                        AS  promocoes_flores
        ,COALESCE(`COMPONENTES`,0)                   AS  promocoes_componentes
        ,COALESCE(`PIPOCAS`,0)                       AS  promocoes_pipocas
        ,COALESCE(`MEIA TRUFA`,0)                    AS  promocoes_meia_trufa
FROM (
SELECT DISTINCT p.data,m.familia_material FROM ds_dev.cvc_val.promocoes p
LEFT JOIN ds_dev.cvc_val.materiais m ON m.codigo_produto=p.codigo_produto
WHERE YEAR(p.data)>={ANO}
ORDER BY p.data
) PIVOT (
  CASE WHEN COUNT(*) > 0 THEN 1 ELSE 0 END FOR familia_material IN (
'PASTAS','OVOS','MARSHMALLOWS','CHOCOARTES','BEBIDAS ALCOOLICAS',
'FIGURAS','CLUSTERS','LIFESTYLE','CARTOES',
'OVINHOS','PIRULITOS','ESFERAS','GRANEIS',
'FLORES','COMPONENTES',
'PIPOCAS','MEIA TRUFA')
)
ORDER BY data''').createOrReplaceTempView("promocoes")

## **FERIADOS NACIONAIS**:

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
WITH CALENDARIO AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), 
  INTERVAL 1 DAY)) AS data
)
SELECT c.data, 
CASE WHEN f.nome_feriado IS NULL THEN 0 ELSE 1 END AS feriado_nacional
FROM CALENDARIO c
LEFT JOIN ds_dev.cvc_val.calendario_anbima_feriados f
ON f.data=c.data
ORDER BY c.data''').createOrReplaceTempView("feriados_nacionais")

## **METRICAS DE MERCADO:**

In [0]:
QUERY_MERCADO = '''WITH RENAME_CODIGO AS (
    SELECT 
        sercodigo                    AS codigo, -- Força Maiúsculo para bater com o STACK
        UPPER(sernome)                    AS metricas_mercado
    FROM ds_dev.cvc_val.metadados_ipea
),
VALORES AS (
    SELECT
      data                                AS data,
      codigo                              AS codigo,
      valor                               AS valor
    FROM ds_dev.cvc_val.medidas_selecionadas_mercado
    LATERAL VIEW STACK(
      31, 
    'SGS12_IBCBR12',       SGS12_IBCBR12,
    'SGS12_IBCBRDESSAZ12', SGS12_IBCBRDESSAZ12,
    'PMC12_VNAALIMN12',    PMC12_VNAALIMN12,
    'PMC12_VRAALIMN12',    PMC12_VRAALIMN12,
    'PMC12_VRSUPN12',      PMC12_VRSUPN12,
    'PMC12_VRSUPNSA12',    PMC12_VRSUPNSA12,
    'BM12_PIB12',          BM12_PIB12,
    'DIMAC_INF1',          DIMAC_INF1,
    'DIMAC_INF2',          DIMAC_INF2,
    'DIMAC_INF3',          DIMAC_INF3,
    'DIMAC_INF4',          DIMAC_INF4,
    'DIMAC_INF5',          DIMAC_INF5,
    'DIMAC_INF6',          DIMAC_INF6,
    'IPP12_IPPC10ATIV12',  IPP12_IPPC10ATIV12,
    'PRECOS12_INPCAB12',   PRECOS12_INPCAB12,
    'PRECOS12_IPCAAB12',   PRECOS12_IPCAAB12,
    'PNADC12_OCUPALOJ12',  PNADC12_OCUPALOJ12,
    'PNADC12_TDESOC12',    PNADC12_TDESOC12,
    'CNC12_PEICRC12',      CNC12_PEICRC12,
    'FCESP12_IIC12',       FCESP12_IIC12,
    'ABRAS12_INVNR12',     ABRAS12_INVNR12,
    'GAC12_PPCTAXAC12',    GAC12_PPCTAXAC12,
    'CNC12_ICF12',         CNC12_ICF12,
    'CNC12_ICFAB12',       CNC12_ICFAB12,
    'CNC12_ICFAC12',       CNC12_ICFAC12,
    'CNC12_ICFAJ12',       CNC12_ICFAJ12,
    'IGP12_IGPMG12',       IGP12_IGPMG12,
    'IGP12_IPCMG12',       IGP12_IPCMG12,
    'PAN12_IVVRG12',       PAN12_IVVRG12,
    'FCESP12_IICF12',      FCESP12_IICF12,
    'ANBIMA12_TJPOUP12',   ANBIMA12_TJPOUP12
    ) AS codigo, valor
),
UNPIVOT_TABLE AS (
    SELECT  
        v.data,
        v.codigo,
        l.lag,
        v.valor,
        add_months(to_date(v.data, 'yyyy-MM-dd'), l.lag) AS date_corrigida
    FROM ds_dev.cvc_val.correlacao_spearman_por_lag_por_canal l
    -- Join com metadados para garantir que a métrica existe
    INNER JOIN RENAME_CODIGO rc 
        ON UPPER(l.metricas_mercado) = rc.metricas_mercado
    -- Join com os valores unpivotados
    INNER JOIN VALORES v USING(codigo)
    WHERE l.canal = '{CANAL}'
)
SELECT 
    date_corrigida                AS data,
    codigo                        AS codigo,
    valor                         AS valor 
FROM UNPIVOT_TABLE
WHERE lag IS NOT NULL 
  AND valor IS NOT NULL'''

### **CANAL** LOJAS

In [0]:
# Processamento SQL massivo de histórico
df_unpivot=spark.sql(QUERY_MERCADO.format(CANAL = 'LOJA'))
df_pivot = df_unpivot.groupBy("data").pivot("codigo").agg({"valor": "sum"})
df_pivot.createOrReplaceTempView("lag_medidas_mercado_lojas")

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
WITH FULL AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), INTERVAL 1 DAY)) AS data_correta
),
CALENDARIO AS (
  SELECT 
    DATE_FORMAT(DATA_CORRETA, 'yyyy-MM') AS data_format,
     data_correta 
  FROM FULL
),
MEDIDAS AS (
  SELECT 
    *,
    DATE_FORMAT(DATA, 'yyyy-MM') AS data_format
  FROM lag_medidas_mercado_lojas
)
SELECT *
FROM CALENDARIO 
LEFT JOIN MEDIDAS USING (data_format)
WHERE YEAR(data) >= {ANO}''').drop('data','data_format').withColumnRenamed('data_correta', 'data').createOrReplaceTempView('medidas_mercado_lojas')

### **CANAL** VENDA DIRETA

In [0]:
# Processamento SQL massivo de histórico
df_unpivot=spark.sql(QUERY_MERCADO.format(CANAL = 'VENDA DIRETA'))
df_pivot = df_unpivot.groupBy("data").pivot("codigo").agg({"valor": "sum"})
df_pivot.createOrReplaceTempView("lag_medidas_mercado_venda_direta")

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
WITH FULL AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), INTERVAL 1 DAY)) AS data_correta
),
CALENDARIO AS (
  SELECT 
    DATE_FORMAT(DATA_CORRETA, 'yyyy-MM') AS data_format,
     data_correta 
  FROM FULL
),
MEDIDAS AS (
  SELECT 
    *,
    DATE_FORMAT(DATA, 'yyyy-MM') AS data_format
  FROM lag_medidas_mercado_venda_direta
)
SELECT *
FROM CALENDARIO 
LEFT JOIN MEDIDAS USING (data_format)
WHERE YEAR(data) >= {ANO}''').drop('data','data_format').withColumnRenamed('data_correta', 'data').createOrReplaceTempView('medidas_mercado_venda_direta')

### **CANAL** E-COMMERCE

In [0]:
# Processamento SQL massivo de histórico
df_unpivot=spark.sql(QUERY_MERCADO.format(CANAL = 'E-COMMERCE'))
df_pivot = df_unpivot.groupBy("data").pivot("codigo").agg({"valor": "sum"})
df_pivot.createOrReplaceTempView("lag_medidas_mercado_ecommerce")

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
WITH FULL AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), INTERVAL 1 DAY)) AS data_correta
),
CALENDARIO AS (
  SELECT 
    DATE_FORMAT(DATA_CORRETA, 'yyyy-MM') AS data_format,
     data_correta 
  FROM FULL
),
MEDIDAS AS (
  SELECT 
    *,
    DATE_FORMAT(DATA, 'yyyy-MM') AS data_format
  FROM lag_medidas_mercado_ecommerce
)
SELECT *
FROM CALENDARIO 
LEFT JOIN MEDIDAS USING (data_format)
WHERE YEAR(data) >= {ANO}''').drop('data','data_format').withColumnRenamed('data_correta', 'data').createOrReplaceTempView('medidas_mercado_ecommerce')

## **Datas Importantes:**

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''WITH CALENDARIO AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), 
  INTERVAL 1 DAY)) AS data
),
EVENTO AS (
SELECT DISTINCT CAST(DATA AS DATE) AS data, nome_feriado FROM ds_dev.cvc_val.calendario_eventos
WHERE nome_feriado IN ('PÁSCOA','NATAL') AND YEAR(data)>={ANO}
ORDER BY data)
SELECT c.data,
CASE WHEN ev.nome_feriado IS NULL THEN 0 ELSE 1 END AS datas_importantes
 FROM CALENDARIO c
LEFT JOIN EVENTO ev USING(data)
ORDER BY c.data''').createOrReplaceTempView("datas_importantes")

# **Feriado Loja:**

In [0]:
# Processamento SQL massivo de histórico
df=spark.sql(f'''WITH LOJA AS (
SELECT DISTINCT codigo_loja, codigo_municipio, tipo_loja, modelo_loja
FROM ds_dev.cvc_val.lojas
WHERE status_loja = 'ATIVO'
)
SELECT  l.codigo_loja,
        l.codigo_municipio,
        l.tipo_loja,
        l.modelo_loja, 
        COALESCE(ev.data,TO_DATE('{DATA_INICIAL}')) AS data,
        ev.tipo_feriado,
        ev.nome_feriado,
        ev.horario_de_inicio,
        ev.horario_de_fim,
       CASE WHEN ev.data IS NOT NULL THEN 1 ELSE 0 END AS contagem
FROM LOJA l
LEFT JOIN ds_dev.cvc_val.calendario_eventos ev USING(codigo_municipio)
WHERE YEAR(ev.data) >={ANO} OR ev.data IS NULL
ORDER BY codigo_loja DESC
''')

pivot_df = df.groupBy("data") \
  .pivot("codigo_loja") \
  .agg(F.when(F.count("contagem") > 0, 1).otherwise(0)) \
  .orderBy("data")

df_calendario=spark.sql(f'''WITH CALENDARIO AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), 
  INTERVAL 1 DAY)) AS data
)
SELECT * FROM CALENDARIO
'''
)
df_resultado = df_calendario.join(pivot_df, on="data", how="left").fillna(0)
df_resultado.orderBy('DATA').createOrReplaceTempView("historico_feriados_loja")

# **Tabela Previsão:** Histórico de Faturamento

In [0]:
# Processamento SQL massivo de histórico
# 1. Carregar a tabela como um DataFrame e aplicar o filtro inicial

df = spark.sql("""
    SELECT 
        f.data,
        f.codigo_loja,
        CAST(f.valor_venda_liquida AS FLOAT) AS valor_venda_liquida
    FROM ds_dev.cvc_val.faturamento_preenchido f
    LEFT JOIN ds_dev.cvc_val.lojas l USING (codigo_loja)
    WHERE 
        l.status_loja = 'ATIVO' 
        AND f.descricao_canal_venda = 'LOJA'
""")

pivot_df = df.groupBy("data") \
             .pivot("codigo_loja") \
             .agg(F.sum("valor_venda_liquida")) \
             .orderBy("data")

pivot_df.createOrReplaceTempView("historico_targuet_loja")

In [0]:
# Processamento SQL massivo de histórico
df = spark.sql("""
    SELECT 
        f.data,
        f.codigo_loja,
        CAST(f.valor_venda_liquida AS FLOAT) AS valor_venda_liquida
    FROM ds_dev.cvc_val.faturamento_preenchido f
    LEFT JOIN ds_dev.cvc_val.lojas l USING(codigo_loja)
    WHERE 
        l.status_loja = 'ATIVO' 
        AND f.descricao_canal_venda = 'VENDA DIRETA'
""")

pivot_df = df.groupBy("data") \
             .pivot("codigo_loja") \
             .agg(F.sum("valor_venda_liquida")) \
             .orderBy("data")

pivot_df.createOrReplaceTempView("historico_targuet_venda_direta")

In [0]:
# Processamento SQL massivo de histórico
df = spark.sql("""
    SELECT 
        f.data,
        f.codigo_loja,
        CAST(f.valor_venda_liquida AS FLOAT) AS valor_venda_liquida
    FROM ds_dev.cvc_val. faturamento_preenchido f
    LEFT JOIN ds_dev.cvc_val.lojas l USING(codigo_loja)
    WHERE 
        l.status_loja = 'ATIVO' 
        AND f.descricao_canal_venda = 'E-COMMERCE'
""")

pivot_df = df.groupBy("data") \
             .pivot("codigo_loja") \
             .agg(F.sum("valor_venda_liquida")) \
             .orderBy("data")

pivot_df.createOrReplaceTempView("historico_targuet_ecommerce")

# **Tabela Suporte:**

In [0]:
spark.table("ds_dev.cvc_val.lojas").createOrReplaceTempView("lojas_fs")

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
WITH CALENDARIO AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), 
  INTERVAL 1 DAY)) AS data
)
SELECT * FROM CALENDARIO
LEFT JOIN campanhas USING (data)
LEFT JOIN promocoes USING (data)
LEFT JOIN feriados_nacionais USING (data)
LEFT JOIN datas_importantes USING (data)
LEFT JOIN medidas_mercado_lojas USING (data)
ORDER BY data''').createOrReplaceTempView("historico_suporte_loja")

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
WITH CALENDARIO AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), 
  INTERVAL 1 DAY)) AS data
)
SELECT * FROM CALENDARIO
LEFT JOIN campanhas USING (data)
LEFT JOIN promocoes USING (data)
LEFT JOIN feriados_nacionais USING (data)
LEFT JOIN datas_importantes USING (data)
LEFT JOIN medidas_mercado_venda_direta USING (data)
ORDER BY data''').createOrReplaceTempView("historico_suporte_venda_direta")

In [0]:
# Processamento SQL massivo de histórico
spark.sql(f'''
WITH CALENDARIO AS (
  SELECT EXPLODE(SEQUENCE(TO_DATE('{DATA_INICIAL}'), CURRENT_DATE(), 
  INTERVAL 1 DAY)) AS data
)
SELECT * FROM CALENDARIO
LEFT JOIN campanhas USING (data)
LEFT JOIN promocoes USING (data)
LEFT JOIN feriados_nacionais USING (data)
LEFT JOIN datas_importantes USING (data)
LEFT JOIN medidas_mercado_ecommerce USING (data)
ORDER BY data ''').createOrReplaceTempView("historico_suporte_ecommerce")

In [0]:
def unpivot_table(df, descricao):
    # 1. Definição das colunas
    id_cols = ["data"] 
    numeric_types = (DoubleType, IntegerType, LongType, DecimalType, FloatType, ShortType)
    
    # Identifica colunas métricas
    metric_cols = [
        f.name for f in df.schema.fields
        if f.name not in id_cols and isinstance(f.dataType, numeric_types)
    ]
    
    if not metric_cols:
        return df # Ou levantar um erro se preferir

    # 2. OTIMIZAÇÃO: Cast em lote (Batch Casting)
    # Criamos uma lista de expressões para o select, evitando o loop de withColumn
    select_exprs = [F.col(c) for c in id_cols] + \
                   [F.col(c).cast("double").alias(c) for c in metric_cols]
    
    df_casted = df.select(*select_exprs)

    # 3. Monta a expressão do stack
    # A lógica do stack permanece a mesma, pois é eficiente para quantidades razoáveis de colunas
    pairs = ", ".join([f"'{c}', `{c}`" for c in metric_cols])
    stack_expr = f"stack({len(metric_cols)}, {pairs}) as ({descricao}, valor)"

    # 4. Aplica Unpivot
    df_unpivot = (
        df_casted.select(*id_cols, F.expr(stack_expr))
                 .filter(F.col("VALOR").isNotNull())
    )
    
    return df_unpivot

# **Tabelas Databricks:**

In [0]:
table_config = {
    'lojas_fs': {'pks': ["codigo_loja"], 'ts': None},
    'historico_suporte_loja': {'pks': ["metricas"], 'ts': "data"},
    'historico_suporte_venda_direta': {'pks': ["metricas"], 'ts': "data"},
    'historico_suporte_ecommerce': {'pks': ["metricas"], 'ts': "data"},
    'historico_targuet_loja': {'pks': ["codigo_loja"], 'ts': "data"},
    'historico_targuet_ecommerce': {'pks': ["codigo_loja"], 'ts': "data"},
    'historico_targuet_venda_direta': {'pks': ["codigo_loja"], 'ts': "data"},
    'historico_feriados_loja': {'pks': ["codigo_loja"], 'ts': "data"}
}

In [0]:
# --- LOOP DE INGESTÃO ---
print(f"🚀 Iniciando ingestão de {len(table_config.keys())} tabelas...")
for table_source in table_config.keys():
    target_table_name = f"ds_dev.cvc_val.{table_source.lower()}"
    if table_source in table_config:
        config = table_config[table_source]
        pks = config['pks']
        timestamp_col = config['ts']
    else:
        print(f"⚠️ Configuração não encontrada para {table_source}. Pulando...")
        continue
    print(f"\n📦 Processando: {table_source} -> {target_table_name}")
    # Verifica configuração
    try:
        # Leitura JDBC
        df_source = unpivot_table(spark.table(table_source), descricao=pks[0])
        # Gravação no Feature Store
        print(f"   💾 Salvando no Feature Store...")
        salvar_feature_table(
            df=df_source,
            table_name_full=target_table_name,
            pk_columns=pks,
            timestamp_col=timestamp_col,
            spark=spark
        )
        print(f"✅ Sucesso: {target_table_name}")
    except Exception as e:
        print(f"❌ Erro ao processar {table_source}: {str(e)}")